In [43]:
from datasets import Dataset

def my_gen():
    f1 = open("labeled_data_sentences.txt", "r")
    f2 = open("labeled_data_labels.txt", "r")

    sentences = f1.readlines()
    labels = f2.readlines()

    for i in range(len(sentences)):
        yield {"text": sentences[i][:len(sentences[i])-1],
               "label": labels[i][:len(labels[i])-1]}

In [44]:
dataset = Dataset.from_generator(my_gen)

Found cached dataset generator (C:/Users/Jonathan Ipe/.cache/huggingface/datasets/generator/default-a52d2eba600dcdeb/0.0.0)


In [45]:
dataset = dataset.class_encode_column("label")

Casting to class labels:   0%|          | 0/40005 [00:00<?, ? examples/s]

In [47]:
dataset[100]

{'text': '¯¯·¯¯À¯¯ ¯¯³¯¯¯¯¯ ¯¯±¯¯¯¯¯ ¯¯°¯¯¯¯¯ ¯¯¯¯ï¸¯¯ ¯¯·¯¯¯¯¯ ¯¯¿¯¯¯¯¯ ¯¯Ï¯¯¯¯¯ ¯¯ï¯¯¯¯¯ ¯¯į¯¯¯¯¯ ¯¯¯°¯¯¯¯ ¯¯¯±¯¯¯¯ ¯¯¯³¯¯¯¯ ¯¯¯±¯¯¯¯ ¯¯¯°¯¯¯¯ ¯¯¯°¯¯¯¯ ¯¯¯³¯ŀ¯¯ ¯¯¯±¯¯¯¯ ¯¯¯±¯¯¯¯ ¯¯¯°¯¯¯¯ ¯¯¯³¯Ï¯¯ ¯¯¯¯¯į¯¯ ¯¯į¯¯³¯¯ ¯¯¯¯¯¿¯¯ ¯¯¯°¯·¯¯ ¯¯¯¯¯Ï¯¯ ¯¯Ï¯¯°¯¯ ¯¯¯¯¯³¯¯ ¯¯ï¯¯±¯¯ ¯¯±¯¯·¯¯ ¯¯·¯ï¯¯¯ ¯¯¯¯¯°¯¯ ¯¯¿¯į¯¯¯ ¯¯¯¯¯±¯¯ ¯¯°¯¿¯¯¯ ¯¯¯¯ï¯¯¯ ¯¯·¯¯°¯¯ ¯¯¯¯į¯¯¯ ¯į¯¯Ï¯¯¯ ¯¯¿¯¯¯¯¯ ¯¯±¯¯¯¯¯ ¯¯¿¯¯¯¯¯ ¯į¯¯Ï¿¯¯ ¯¯¿¯¯¯¯¯ ¯¯±¯¯¯¯¯ ¯¯¿¯¯¯¯¯ ¯ï¯¯ïÏ¯¯ ¯¯·¯¯¯¯¯ ¯¯°¯¯¯¯¯ ¯¯·¯¯¯¯¯ ¯Ï¯¯įï¯¯ ¯¯¿¯¯¯¯¯ ¯¯±¯¯¯¯¯ ¯¯¿¯¯¯¯¯ ¯¯°¯°°¯¯ ¯¯Ï¯¯¯¯¯ ¯¯·¯¯¯¯¯ ¯¯Ï¯¯¯¯¯ ¯¯°¯¯¯¯¯ ¯¯Ï¯¯¯¯¯ ¯¯·¯ï¯¯¯ ¯¯Ï¯¿¯¯¯ ¯¯°¯¯¯¯¯ ¯¯ï¯¯¯¯¯',
 'label': 5}

In [48]:
from transformers import AutoTokenizer, RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained("BPEtokenizer", max_len=512)

In [49]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [50]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/40005 [00:00<?, ? examples/s]

In [51]:
small_train_dataset = tokenized_datasets.shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets.shuffle(seed=42).select(range(1000, 1500))

Loading cached shuffled indices for dataset at C:\Users\Jonathan Ipe\.cache\huggingface\datasets\generator\default-a52d2eba600dcdeb\0.0.0\cache-b382376c12ced46f.arrow


Training

In [52]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("BERT-Pretrained", num_labels=9)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at BERT-Pretrained and are newly initialized: ['classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [53]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [54]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [55]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [56]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", 
                                  evaluation_strategy="epoch")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [57]:
trainer.train()

  0%|          | 0/375 [00:00<?, ?it/s]

  0%|          | 0/63 [00:00<?, ?it/s]

{'eval_loss': 2.1849923133850098, 'eval_accuracy': 0.144, 'eval_runtime': 11.2557, 'eval_samples_per_second': 44.422, 'eval_steps_per_second': 5.597, 'epoch': 1.0}


  0%|          | 0/63 [00:00<?, ?it/s]

{'eval_loss': 2.213439702987671, 'eval_accuracy': 0.168, 'eval_runtime': 11.2847, 'eval_samples_per_second': 44.308, 'eval_steps_per_second': 5.583, 'epoch': 2.0}


  0%|          | 0/63 [00:00<?, ?it/s]

{'eval_loss': 2.1105146408081055, 'eval_accuracy': 0.21, 'eval_runtime': 10.9829, 'eval_samples_per_second': 45.525, 'eval_steps_per_second': 5.736, 'epoch': 3.0}
{'train_runtime': 455.0712, 'train_samples_per_second': 6.592, 'train_steps_per_second': 0.824, 'train_loss': 2.179404947916667, 'epoch': 3.0}


TrainOutput(global_step=375, training_loss=2.179404947916667, metrics={'train_runtime': 455.0712, 'train_samples_per_second': 6.592, 'train_steps_per_second': 0.824, 'train_loss': 2.179404947916667, 'epoch': 3.0})